In [4]:
import requests
from getpass import getpass
from scraper import WikiSearch
from wikienv import WikiEnv
from wrappers import HotPotQAWrapper, LoggingWrapper
from bot_interface import Bot

API_KEY = getpass('Enter your OpenAI key: ')

In [5]:
AGENT = 'teaLLM'
# AGENT = 'ReAct'
# AGENT = 'CoT'

In [6]:
ENVS = {'teaLLM' : WikiSearch(api_key=API_KEY),
        'ReAct'  : WikiEnv()}
env = ENVS[AGENT]
env = HotPotQAWrapper(env, split='dev')
env = LoggingWrapper(env)
bot = Bot(api_key=API_KEY)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

In [23]:
folder = './prompts/'
prompt_file = folder + f'{AGENT}.txt'
with open(prompt_file, 'r') as f:
    webthink_prompt = '\n'.join(f.readlines())
    
def webthink(question=None, idx=None, bot=bot, prompt=webthink_prompt, to_print=True):
    if not (idx is None):
        question = env.reset(idx=idx)
    else:
        env.reset()
    if to_print and not idx is None: print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = bot(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = bot(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
            
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print: print(step_str)
        if done: break
    if not done:
        finish_str = 'finish | NO ANSWER' if AGENT == 'teaLLM' else 'finish[NO ANSWER]'
        obs, r, done, info = step(env, finish_str)
        
    if not (idx is None) and to_print: print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

## Custom questions

In [24]:
# teamLLM should generally provide more comprehensive and informative answers
# answer: Ada Lovelace architecture, third-generation RT cores, fourth-generation Tensor Cores
_ = webthink(question='Why is the RTX 4090 so fast?',
             to_print=True)

Thought 1: I need to find information on the speed of the RTX 4090 graphics card. I should start by searching for the RTX 4090 and its performance details.

Action 1: search | RTX 4090 | what makes the RTX 4090 fast? | speed
Observation 1: The RTX 4090 is fast due to being based on Nvidia's Ada Lovelace architecture and featuring third-generation RT cores for hardware-accelerated ray tracing (RTX) and fourth-generation deep-learning-focused Tensor Cores. 

Thought 2: The reason the RTX 4090 is so fast is because of Nvidia's Ada Lovelace architecture and the inclusion of third-generation RT cores for hardware-accelerated ray tracing and fourth-generation Tensor Cores for deep learning tasks. This makes the RTX 4090 a powerful graphics card for performance-driven tasks like gaming and rendering. I can provide this information as the answer.
Action 2: finish | Ada Lovelace architecture, third-generation RT cores, fourth-generation Tensor Cores
Observation 2: Episode finished, reward = 0



## HotPotQA

In [28]:
import time, json
from tqdm import tqdm

idxs = list(range(7405))
rs = []
infos = []
old_time = time.time()
BEGIN = 0
END = 14
for i in tqdm(idxs[BEGIN:END]):
# for i in tqdm(idxs[758:1000]):
    print(f'INDEX = {i}')
    r, info = webthink(idx=i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

with open(f'{AGENT}_temp.json', 'w') as f:
    json.dump(infos, f)

  0%|          | 0/14 [00:00<?, ?it/s]

INDEX = 0
0 Question: Were Scott Derrickson and Ed Wood of the same nationality?
Thought 1: Let's search for Scott Derrickson and Ed Wood to find out if they are of the same nationality.
Action 1: search | Scott Derrickson | nationality of Scott Derrickson | nationality
Observation 1: Scott Derrickson is an American filmmaker known for horror films like The Exorcism of Emily Rose and Sinister, as well as the superhero film Doctor Strange. He grew up in Denver, Colorado, and graduated from Biola University and USC School of Cinema-Television.

Thought 2: The search for Scott Derrickson did not indicate his nationality, so I will search for Ed Wood to see if they share the same nationality. 
Action 2: search | Ed Wood | nationality of Ed Wood | nationality
Observation 2: Edward Davis Wood Jr. was an American filmmaker, actor, screenwriter, and pulp novelist. He was born in Poughkeepsie, New York, in 1924. Throughout the 1950s, he directed low-budget science fiction, crime, and horror fil

  7%|▋         | 1/14 [00:09<02:02,  9.41s/it]

Thought 3: Since both Scott Derrickson and Ed Wood are American filmmakers, they are of the same nationality. The question can be answered now.
Action 3: finish | Yes
Observation 3: Episode finished, reward = 1


1 1 1.0 9.411827802658081
-----------

INDEX = 1
1 Question: What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
Thought 1: I need to find the government position held by the woman who played Corliss Archer in Kiss and Tell. I should start by searching for details about the film and the actress who portrayed Corliss Archer.
Action 1: search | Kiss and Tell film | who portrayed Corliss Archer in the film Kiss and Tell? | Corliss Archer
Observation 1: There are no sentences in the provided text related to the question about who portrayed Corliss Archer in the film Kiss and Tell. The text does not contain any information on specific actors or characters in movies.

Thought 2: Since I couldn't find the actress who played Corliss A

 14%|█▍        | 2/14 [00:18<01:52,  9.35s/it]

Thought 4: Shirley Temple portrayed Corliss Archer in the 1945 film "Kiss and Tell". The original question asked about the government position held by the woman who played Corliss Archer. Shirley Temple later went on to hold the position of U.S. Ambassador to Ghana and Czechoslovakia. I should finish with this answer.
Action 4: finish | U.S. Ambassador
Observation 4: Episode finished, reward = 0


1 2 0.5 9.35993492603302
-----------

INDEX = 2
2 Question: What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and alien species?
Thought 1: I need to search for a science fantasy young adult series that is told in the first person and has companion books about enslaved worlds and alien species. I should start by searching for science fantasy young adult series.

Action 1: search | science fantasy young adult series | what series is told in first person and has companion books about enslaved worlds and alien spe

 14%|█▍        | 2/14 [00:43<04:22, 21.91s/it]

Thought 7: I have not found the information I was looking for in the search for "The Mortal Instruments." I should continue searching for other series mentioned in the similar articles list that might fit the criteria.

Action 7: search | Maximum Ride | do the companion books narrate the stories of enslaved worlds and alien species? | enslaved worlds, alien species
Observation 7: There is no mention of companion books narrating the stories of enslaved worlds and alien species in the provided text. The text mainly focuses on the adventures of Maximum Ride and her family, the Flock, who are human-avian hybrids created in a lab called The School, and their battles against various obstacles and enemies.



KeyError: 'em'